In [78]:
import pandas as pd
import os
from datetime import datetime

DATA_ROOT = os.path.join(os.getcwd(), 'data')
STAGING_ROOT = os.path.join(os.getcwd(), 'staging')

# Import The Joy Of Painiting - Colors Used
df_colors = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painiting - Colors Used'))
# create episode ID to join to Subject Matter
df_colors['episode_id'] = df_colors.apply(
    lambda r: f"S{r['season']:02}E{r['episode']:02}", axis=1)

# Import The Joy Of Painiting - Subject Matter
df_subjects = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painiting - Subject Matter'))
# remove surrounding quotes from TITLE
df_subjects['TITLE'] = df_subjects.apply(lambda r: r['TITLE'][1:-1], axis=1)

# Import The Joy Of Painting - Episode Dates
df_dates = pd.read_csv(os.path.join(
    DATA_ROOT, 'The Joy Of Painting - Episode Dates'), delimiter="\t", header=None, names=['src'])


In [79]:
# parse df_dates (not actually CSV-- parse each row of file as col in df)
df_dates['date'] = df_dates.apply(
    lambda r: r['src'][r['src'].find('(')+1:r['src'].find(')')].strip(), axis=1)
# use parsed date to split src & get title
df_dates['title'] = df_dates.apply(
    lambda r: r['src'].split(r['date'])[0][:-1].strip(), axis=1
)
# use parsed date to split src & get note (after title)
df_dates['note'] = df_dates.apply(
    lambda r: r['src'].split(r['date'])[1][1:-1].strip(), axis=1
)
# convert date string to date object to sort
df_dates['date'] = df_dates.apply(
    lambda r: datetime.strptime(r['date'], '%B %d, %Y'), axis=1)
# pull EPISODE and TITLE into new df to join to df_dates
df_episodeIds = df_subjects[['EPISODE', 'TITLE']].sort_values('EPISODE')
# join sorted datasets
df_dates = df_dates.sort_values('date').join(df_episodeIds)
# check titles are aligned
pd.concat([df_dates.head(10), df_dates.tail(10)])

,src,date,title,note,EPISODE,TITLE
0,"A Walk in the Woods (January 11, 1983)",1983-01-11,A Walk in the Woods,,S01E01,A WALK IN THE WOODS
1,"Mount McKinley (January 11, 1983)",1983-01-11,Mount McKinley,,S01E02,MT. MCKINLEY
2,"Ebony Sunset (January 18, 1983)",1983-01-18,Ebony Sunset,,S01E03,EBONY SUNSET
3,"Winter Mist (January 25, 1983)",1983-01-25,Winter Mist,,S01E04,WINTER MIST
4,"Quiet Stream (February 1, 1983)",1983-02-01,Quiet Stream,,S01E05,QUIET STREAM
5,"Winter Moon (February 8, 1983)",1983-02-08,Winter Moon,,S01E06,WINTER MOON
6,"Autumn Mountains (February 15, 1983)",1983-02-15,Autumn Mountains,,S01E07,AUTUMN MOUNTAINS
7,"Peaceful Valley (February 22, 1983)",1983-02-22,Peaceful Valley,,S01E08,PEACEFUL VALLEY
8,"Seascape (March 1, 1983)",1983-03-01,Seascape,,S01E09,SEASCAPE
9,"Mountain Lake (March 8, 1983)",1983-03-08,Mountain Lake,,S01E10,MOUNTAIN LAKE


In [80]:
from ast import literal_eval
# get data needed for colors into new df
df_episode_colors = df_colors[['episode_id', 'colors', 'color_hex']]
# convert list columns
df_episode_colors['colors'] = df_episode_colors['colors'].apply(literal_eval)
df_episode_colors['color_hex'] = df_episode_colors['color_hex'].apply(
    literal_eval)
# convert colors list to list of dicts {color: '...', hex: '...'}
df_episode_colors['colors'] = df_episode_colors.apply(
    lambda r: [{"color": k, "hex": v} for k, v in dict(zip(r['colors'], r['color_hex'])).items()], axis=1)

# create a row for every color item in colors list column
df_episode_colors = df_episode_colors.drop(
    columns=['color_hex']).explode('colors')

# split dict back into columns
df_episode_colors['color_hex'] = df_episode_colors['colors'].apply(
    lambda c: c['hex'])
df_episode_colors['color'] = df_episode_colors['colors'].apply(
    lambda c: c['color'])

df_episode_colors['color_id'] = df_episode_colors['color_hex'].apply(
    lambda c: c[1:])

df_unique_colors = df_episode_colors[[
    'color_id', 'color', 'color_hex']].drop_duplicates().rename(columns={"color_id": 'id'})
df_episode_colors = df_episode_colors[[
    'color_id', 'episode_id']].drop_duplicates()

df_episode_colors.to_csv(os.path.join(
    STAGING_ROOT, 'episode_colors.csv'), index=False)
df_unique_colors.to_csv(os.path.join(STAGING_ROOT, 'colors.csv'), index=False)


<ipython-input-80-e2afc20219de>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_episode_colors['colors'] = df_episode_colors['colors'].apply(literal_eval)
<ipython-input-80-e2afc20219de>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_episode_colors['color_hex'] = df_episode_colors['color_hex'].apply(
<ipython-input-80-e2afc20219de>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

KeyError: "['hex'] not in index"

In [ ]:
# get episodes DF
df_episodes = df_dates.set_index('EPISODE').join(df_colors.set_index('episode_id', drop=False))[
    ['episode_id', 'season', 'episode', 'date', 'title', 'painting_index', 'img_src', 'painting_title', 'youtube_src']]
df_episodes = df_episodes.rename(columns={"episode_id": 'id'})
df_episodes.head()

df_episodes.to_csv(os.path.join(STAGING_ROOT, 'episodes.csv'), index=False)


In [ ]:
# unpivot subjects into df_episode_subjects
df_episode_subjects = df_subjects.melt(id_vars=['EPISODE', 'TITLE'],
                                       var_name='subject', value_name='has_subject')
# drop subjects not in episodes
df_episode_subjects = df_episode_subjects[df_episode_subjects['has_subject'] == 1]
# get unique subjects
df_unique_subjects = df_episode_subjects[['subject']].drop_duplicates()
# get friendly name for unique subject
df_unique_subjects['name'] = df_unique_subjects.apply(
    lambda r: r['subject'].replace('_', ' ').title(), axis=1)
df_unique_subjects = df_unique_subjects.rename(columns={"subject": "id"})
# df_episode_subjects to episodeId & subjectId
df_episode_subjects = df_episode_subjects[['EPISODE', 'subject']].drop_duplicates(
).rename(columns={"EPISODE": "episode_id", "subject": "subject_id"})

df_unique_subjects.to_csv(os.path.join(STAGING_ROOT, 'subjects.csv'), index=False)
df_episode_subjects.to_csv(os.path.join(STAGING_ROOT, 'episode_subjects.csv'), index=False)
